In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -r "/content/drive/MyDrive/Verizon ML Project/requirements.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.1/874.1 kB 22.9 MB/s eta 0:00:00


In [5]:
from ultralytics import YOLO
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from google.colab.patches import cv2_imshow

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
PROJECT_DIR = "/content/drive/MyDrive/Verizon ML Project"
OUTPUT_DIR = os.path.join(PROJECT_DIR, "output")
YAML_PATH = os.path.join(PROJECT_DIR, "dataset.yaml")

CACHE_MODE = 'disk'

## Tune Manually

1. Loop through parameters
2. For every value in the parameter's given list,
   
   a. Load a new model.
   
   b. Test the value for ten epochs. Make sure you specify the `project` and `name` parameters so that you can compare `results.csv`.

   c. Manually evaluate the precision (prioritize minimal false positives).

   d. Record the value w/ best precision score.

   NOTE: If the parameter performs data augmentation, make sure the value is reasonable (e.g. we shouldn't adjust hue with `hsv_h=1.0`)

This should take 1-2 hours.

#### TRAIN WITH A SMALLER MODEL TO REDUCE TRAINING TIME

In [ ]:
"""
Author:       Chris Taguba
Date:         9/22/24
Description:  This script performs hyperparameter tuning using the YOLOv8 model
              on various parameters like 'hsv_h', 'hsv_s', 'degrees', etc. The
              model is trained for 10 epochs for each parameter value with
              specific configurations.
"""

all_params = {
    'hsv_h': np.linspace(0.0, 1.0, 9),
    'hsv_s': np.linspace(0.0, 1.0, 9),
    'hsv_v': np.linspace(0.0, 1.0, 9),
    'degrees': np.linspace(-20.0, 20.0, 9),
    'translate': np.linspace(0.0, 1.0, 9),
    'scale': np.linspace(0.0, 1.0, 9),
    'shear': np.linspace(-4.0, 4.0, 9),
    'perspective': np.linspace(0.0, 0.001, 9),
}

for key in all_params:

    for value in all_params[key]:

        # this prevents RepresenterError
        if value is None:
            continue
        if isinstance(value, float):
            value = str(value)

        print(f"\nTraining with {key}={value}\n")

        # each new hyperparameter value starts w/ the same model
        # use small model to reduce training time
        model = YOLO(os.path.join(OUTPUT_DIR, "v8s-default", "weights", "best.pt"))

        results = model.train(
            data=YAML_PATH,

            epochs=10, # train less to cover larger search space in shorter time
            close_mosaic=2,
            batch=16,
            imgsz=640,

            project=OUTPUT_DIR,
            name=f"v8s_{key}-{value}",

            save=False, # do not save since xl .pt models are >100 MB each
            cache=CACHE_MODE,
            device=0,
            pretrained=True,
            plots=False,
            exist_ok=True,

            # dynamically insert training keywords
            **{key:float(value)}
        )


Training with degrees=-20.0

Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Verizon ML Project/output/v8s-default/weights/best.pt, data=/content/drive/MyDrive/Verizon ML Project/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=False, save_period=-1, cache=disk, device=0, workers=8, project=/content/drive/MyDrive/Verizon ML Project/output, name=v8s_degrees--20.0, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=2, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_

train: Scanning /content/drive/.shortcut-targets-by-id/13_J0kDa9eHag4lJahP4rlro1gZ-tsaD9/Verizon ML Project/dataset/labels/train.cache... 200 images, 1 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
train: Caching images (0.4GB Disk): 100%|██████████| 200/200 [00:00<00:00, 2259.81it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/.shortcut-targets-by-id/13_J0kDa9eHag4lJahP4rlro1gZ-tsaD9/Verizon ML Project/dataset/labels/val.cache... 57 images, 0 backgrounds, 0 corrupt: 100%|██████████| 57/57 [00:00<?, ?it/s]
val: Caching images (0.1GB Disk): 100%|██████████| 57/57 [00:00<00:00, 718.14it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000345, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/Verizon ML Project/output/v8s_degrees--20.0
Starting training for 10 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/.shortcut-targets-by-id/13_J0kDa9eHag4lJahP4rlro1gZ-tsaD9/Verizon ML Project/dataset/labels/val.cache... 57 images, 0 backgrounds, 0 corrupt: 100%|██████████| 57/57 [00:00<?, ?it/s]
val: Caching images (0.1GB Disk): 100%|██████████| 57/57 [00:00<00:00, 190.09it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000345, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/Verizon ML Project/output/v8s_perspective-0.000875
Starting training for 10 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      7.42G     0.9103      0.988      1.526         23        640: 100%|██████████| 13/13 [00:08<00:00,  1.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.19it/s]


                   all         57         57      0.806      0.859       0.89      0.855


## Compare Precision Across Each Value
Function allows you to compare values by printing out the following:
- Value w/ best precision (and cls_loss at best precision)
- Value w/ best cls_loss (and precision at cls_loss)

After finishing tuning for a hyperparameter, set up the directories like this:

---- hsv_h-tune (`p_dir`)

-------- v8s_hsv_h-0.0 (`subdir`)

-------- v8s_hsv_h-0.125

-------- v8s_hsv_h-0.25

-------- ...

In [44]:
# List of hyperparameters to tune
params = [
    'hsv_h-tune',
    'hsv_s-tune',
    'hsv_v-tune',
    'degrees-tune',
    'translate-tune',
    'scale-tune',
    'shear-tune',
    'perspective-tune',
]

# Loop through each hyperparameter
for p in params:

    # Directory storing all tuning experiments for the hyperparameter
    p_dir = os.path.join(OUTPUT_DIR, p)

    # Initialize best metrics
    max_prec = 0
    min_cls_loss = float('inf')
    best_prec_dir = ""         # Directory with the highest precision
    best_cls_dir = ""          # Directory with the lowest classification loss
    prec_at_min_cls = ""       # Precision at the lowest classification loss
    cls_at_max_prec = ""       # Classification loss at the highest precision

    # Loop through subdirectories for different values of the hyperparameter
    for val_dir in os.listdir(p_dir):

        # Try to read the results CSV for the current tuning run
        try:
            df = pd.read_csv(os.path.join(p_dir, val_dir, 'results.csv'))
        except:
            continue  # Skip if the file is missing or unreadable

        # Extract max precision and min classification loss
        prec = df['   metrics/precision(B)'].max()
        cls_loss = df['           val/cls_loss'].min()

        # Update best precision if the current run has a higher value
        if prec > max_prec:
            max_prec = prec
            best_prec_dir = val_dir

            max_prec_idx = df['   metrics/precision(B)'].idxmax()
            cls_at_max_prec = df.iloc[max_prec_idx]['           val/cls_loss']

        # Update best classification loss if the current run has a lower value
        if cls_loss < min_cls_loss:
            min_cls_loss = cls_loss
            best_cls_dir = val_dir

            min_cls_idx = df['           val/cls_loss'].idxmin()
            prec_at_min_cls = df.iloc[min_cls_idx]['   metrics/precision(B)']

    # Print the results for the current hyperparameter
    print(p)
    print(f"Max precision: {max_prec} in {best_prec_dir} (Class loss at max precision: {cls_at_max_prec})")
    print(f"Min classification loss: {min_cls_loss} in {best_cls_dir} (Precision at min classification loss: {prec_at_min_cls})")
    print()


hsv_h-tune
max_prec: 0.97777 in v8s_hsv_h-0.375 (cls_loss at max_prec 0.33726)
min_cls_loss: 0.26228 in v8s_hsv_h-1.0 (prec at min_cls_loss: 0.94655)

hsv_s-tune
max_prec: 0.97456 in v8s_hsv_s-0.375 (cls_loss at max_prec 0.31644)
min_cls_loss: 0.25215 in v8s_hsv_s-0.625 (prec at min_cls_loss: 0.94679)

hsv_v-tune
max_prec: 0.95906 in v8s_hsv_v-0.25 (cls_loss at max_prec 0.34085)
min_cls_loss: 0.31971 in v8s_hsv_v-0.5 (prec at min_cls_loss: 0.9294)

degrees-tune
max_prec: 0.96678 in v8s_degrees-0.0 (cls_loss at max_prec 0.36804)
min_cls_loss: 0.35931 in v8s_degrees-0.0 (prec at min_cls_loss: 0.86159)

translate-tune
max_prec: 0.94407 in v8s_translate-0.75 (cls_loss at max_prec 0.5928)
min_cls_loss: 0.35603 in v8s_translate-0.375 (prec at min_cls_loss: 0.93844)

scale-tune
max_prec: 0.96678 in v8s_scale-0.5 (cls_loss at max_prec 0.36804)
min_cls_loss: 0.355 in v8s_scale-0.125 (prec at min_cls_loss: 0.89814)

shear-tune
max_prec: 0.96678 in v8s_shear-0.0 (cls_loss at max_prec 0.36804)
min

## Best Hyperparameters:

In [ ]:
hsv_h = 0.375
hsv_s = 0.375
hsv_v = 0.25
degrees = 0.0
translate = 0.375
scale = 0.5
shear = 0.0
perspective = 0.0